## MS20 Task 5 Server (Py3)
<p> On Jupyter, open and run the server and clients in different windows.  
Run the Server first. You should get the message: "Server started and waiting for clients."
Switching to another window will leave the Server running.    The SP-Client is intended to demonstrate the messages your smartphone may send to the Server when you develop your Android software in Task 5. It therefore allows you to see what the Server does.  The SP-Client tries most of the message commands.  The BarryBot client is very similar, and can be made to send messages to the SP_Client and vice-versa.  After trying out commands automatically, both clients allow you to enter messages directly from the keyboard. 
When using the SP_Client and BarryBot, it is necessary to press return to check for messages from another client.   
Feel free to make changes to any of this Py3 software.
When debugging any changes, you will probably need to restart the kernel each time you run the Server.
After restarting the Kernel, wait until you see a little blue "Kernal Ready" flag before running the Server again<p/>
 <p> The Server has been written in a very simple way so that you can understand it and modify it if necessary.  
The heart of it is a simple database implemented using four Python Lists: whoDB, inviteDB, connectDB and socketDB.  We have made these global variables, though this may be unnecessary.</p>
<p>   whoDB is List of clients that are currently registered </p>
<p>   invitedDB is a List for storing which (if any) client is being invited to connect by your client. </p>
<p>   connectedDB is a List for storing which (if any) other client is being ivited to connect by each client.</p>
<p>   socketDB is a List of sockets for the clients that are currently registered and listed in whoDB.</p>
<p>To make debugging easier, there are two 'dummy clients' called Dummy1 and Dummy2.  Clients can connect to these dummy clients as if they were real ones.  To support their activities, there are two strings: Dummy1_buf and  Dummy2_buf which must be declared as global variables.  The dummy clients are assumed to be connected and registered all the time.
The Server is multi-threaded.  It opens a new thread for each new client that connects.  So, if you have SP_Client and BarryBot_Client connected, there will be three threads running: the main thread, and a thread for each of the two clients.
Essentially, each thread is an independent copy of the Server software, with all its methods.  The threads are linked only by the global variables. To avoid conficts when different threads try to write to or read from the global variables, there is a 'mutex' lock mechanism. 
All this mechanism needs is a 'lock' object in the main thread.  All threads will refer to this 'lock' object before they try to read from or write to the global variables.  Whichever thread gets to the lock first will have exclusive access to the global variables.  The others will have to wait until the first thread has finished and released the lock. This makes sure that threads do not try to access the global variables at the same time
The following commands are as in the Task 4 version:
        REGISTER clientuserName 
        UNREGISTER clientuserName
        INVITE clientuserName
        UNINVITE clientuserName
        FORCEINVITE clientuserName       
        ACCEPT clientuserName
        DECLINE clientuserName
        MSG clientuserName Any_message
        QUIT clientuserName
        WHO
This Task 5 version of the server introduces 3 new messages:
        LSMSG clientuserName
        BSMSG clientuserName
        BSPARAMS bit_error_Prob frame_loss_Prob burstiness framelength
See Laboratory Script for details

In [ ]:
# New MS20_Task4_Server_P3 
import socket 
import _thread
import time
import numpy as np
#--- --- ---- -------- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
# Here is the 'on_new_client' method that is run independently for each thread:
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
def on_new_client(clientsocket, addr, lock, tid):
    
    global whoDB, invitedDB, connectedDB, socketDB  # Cental database lists (maybe global is unnecessary)
    global Dummy1_buf, Dummy2_buf   # Central database strings for the Dummy clients (must be global) 
    
    # Here are the methods which service the commands.  cuN is 'clientuserName', msg is message
    def register(cuN, msg):
        global whoDB, invitedDB, connectedDB, socketDB
        #print("lock status is ",lock.locked())
        #lock.acquire()
        #print("lock status is ",lock.locked())        
        if cuN !="" :
            send_to_client("Cannot register again before unregistering first ")
            new_cuN = cuN
            return new_cuN
        new_cuN = msg[9:].rstrip()  # rstrip removes spaces after username
        print("Client username to be registered: ",cuN)
        if new_cuN in whoDB:
            send_to_client("Error: clientuserName already in use.  Choose another")
            new_cuN = cuN
            return new_cuN
        elif new_cuN.rstrip() == "":
            send_to_client("Error: a client user Name must be given") # Only whitespace here
            new_cuN = cuN
            return new_cuN    
        send_to_client( "Your userName "+new_cuN+" is now registered with the server")
        # Register the clientuserName in the whodB data-base
        whoDB.append(new_cuN) ;
        invitedDB.append("none") ; connectedDB.append("none")  # Nobody invited or connected yet
        socketDB.append(clientsocket) # Records address & port of connection for new_cuN
        #lock.release()
        #print("lock status is ",lock.locked()) 
        return new_cuN
    
    def unregister(my_cuN):
        global whoDB, invitedDB, connectedDB, socketDB
        # Unregisters my_cuN, but keeps connection open.  Opposite to REGISTER.
        if (my_cuN not in whoDB): 
            send_to_client("You are not registered.")
            new_cuN = ""
            return new_cuN
        send_to_any_client ("Your userName "+my_cuN+" will be unregistered",my_cuN)
        i = whoDB.index(my_cuN) 
        del whoDB[i]
        del invitedDB[i]
        del connectedDB[i]
        del socketDB[i]
        new_cuN = ""
        return new_cuN     
    
    def invite(my_cuN, msg):
        # InvitedcuN receives INVITEDBY my_cuN
        global whoDB, invitedDB, connectedDB, socketDB
        if my_cuN=="" or my_cuN not in whoDB: 
            send_to_client("Your userName "+my_cuN+" is not registered")
            return
        invited_cuN = msg[7:].rstrip() # rstrip() removes any spaces on the end of a string
        if invited_cuN not in whoDB:
            send_to_client("Invited userName "+invited_cuN+" is not registered")
            return
        # Check if there is already a channel open 
        i = whoDB.index(my_cuN) # index of my_cuN in whoDB database
        j = whoDB.index(invited_cuN) # index of invited_cuN in whoDB database
        if connectedDB[i]==invited_cuN or connectedDB[j]==my_cuN:
            send_to_client("Connection between"+my_cuN+"and "+invited_cuN+" is already open")
            return
        # Otherwise send an invitation (works for real clients & dummies):
        invitedDB[i] = invited_cuN    # Sorry, only one allowed.
        
        if (invited_cuN != "Dummy1") and (invited_cuN != "Dummy2"):
            send_to_any_client("INVITEDBY "+my_cuN , invited_cuN )
            send_to_any_client("INVITED "+invited_cuN , my_cuN )
        else: 
            # Dummy clients always ACCEPT and try to connect to my_cuN (no other)
            if connectedDB[i] == "none": 
                print(invited_cuN," is automatically ACCEPTING a connection to ", my_cuN)
                connectedDB[i] = invited_cuN
                send_to_any_client("INVITED "+invited_cuN+" and "+invited_cuN+" auto-accepted", my_cuN )
            else: 
                print(invited_cuN, "cannot connect to ",my_cuN, " as ",my_cuN," is busy")
                send_to_any_client("INVITED "+invited_cuN+" but connection cannot be made", my_cuN )       
        return

    def uninvite(my_cuN):
        global invitedDB, connectedDB
        # Cancels any INVITE made by my_cuN & cancels any connection to myCun.
        # Therefore uninvite can be used for disconnect also
        if (my_cuN not in whoDB): 
            send_to_client("Your client username is not registered")
            return
        i = whoDB.index(my_cuN) 
        inv_uN= invitedDB[i]; con_uN = connectedDB[i] 
        invitedDB[i] = "none"; connectedDB[i] = "none"
        if inv_uN in whoDB: 
            j = whoDB.index(inv_uN); 
            invitedDB[j] = "none"
        if con_uN in whoDB: 
            j = whoDB.index(con_uN); 
            connectedDB[j] = "none"   
        send_to_any_client("All users uninvited and disconnected from "+my_cuN, my_cuN)
        return
    
    def forceinvite(my_cuN):
        global invitedDB, connectedDB
        # For debugging.  Forces Dummy1 to send an invite to my_cuN
        # When any user sends the command INVITE, the receiver gets INVITEDBY         
        if my_cuN not in whoDB:
            send_to_client("Error: Client userName not registered")
            return     
        send_to_client("INVITEDBY Dummy1")
        print("Dummy1 is inviting ",my_cuN)
        j = whoDB.index("Dummy1")
        invitedDB[j] = my_cuN
        return                      
                  
    def accept(my_cuN, msg):
        global whoDB, invitedDB, connectedDB, socketDB
        # If an invite is accepted, the connection is made automatically
        acceptwhom = msg[7 :].rstrip()
        print(my_cuN," is sending ACCEPT to ", acceptwhom)
        if (my_cuN not in whoDB): 
            send_to_any_client("Username "+my_cuN+" is not registered",my_cuN)
            return
        # Check if acceptwho has registered and has sent an invite to clientuserName
        if (acceptwhom not in whoDB):
            send_to_any_client("Username "+acceptwhom+" is not registered",my_cuN)
            return
        i = whoDB.index(my_cuN)  ;  j = whoDB.index(acceptwhom) ; 
        if (invitedDB[j] != my_cuN):
            send_to_any_client("Error: "+acceptwhom+" has not invited you, sorry",my_cuN)
            return
        if (connectedDB[i]=="none") and (connectedDB [j]=="none") :
            connectedDB[i] = acceptwhom ;  connectedDB[j] = my_cuN;
        else:
            send_to_client("Error: Connection already exists")
            print("connectedDB[i] for ",my_cuN," =", connectedDB[i])
            print("connectedDB[j] for ",acceptwhom," =", connectedDB[j])
            return
        send_to_any_client(my_cuN+" accepted your invite and a connection has been made",acceptwhom)
        send_to_any_client("Invite from "+acceptwhom+" has been accepted & connection made", my_cuN)
        invitedDB[j]="none"    # Remove invite after accepting
        return
                           
    def decline(my_cuN, msg): 
        global whoDB, invitedDB, connectedDB, socketDB
        declinewhom = msg[8 :].rstrip()
        print(my_cuN," is sending DECLINE to ", declinewhom)
        if (my_cuN not in whoDB): 
            send_to_any_client("Your username "+my_cuN+" is not registered",my_cuN)
            return
        # Check if declinewho has registered and has sent an invite to my_cuN
        if (declinewhom not in whoDB):
            send_to_any_client("Error: "+declinewhom+" is not registered",my_cuN)
            return
        i = whoDB.index(my_cuN)  ;  j = whoDB.index(declinewhom)  
        if (invitedDB[j] != my_cuN):
            send_to_any_client("Error: "+declinewhom+" has not invited you, sorry", my_cuN)
            return
        else: 
            connectedDB[j]=="none"  # Cancels invitation from declinewho
            # Keep connectedDB[i] unchanged since my_cuN might be inviting somebody else
            send_to_any_client("Invite from "+declinewhom+" has been declined.",my_cuN)
            send_to_any_client("Invite to "+my_cuN+" has been declined.",declinewhom)
        
        invitedDB[j]="none"    #Remove invite after declining
        return        
    
    def recv_from_Dummy1():
        if Dummy1_buf[0:7] =="MSGFROM":
            msg ="MSGFROM Dummy1 I can only send you this fixed test message"
        elif Dummy1_buf[0:9] == "BSMSGFROM":
            msg = "BSMSGFROM Dummy1 S 10000000000000000000000000000001" # Need something better
        elif Dummy1_buf[0:9] == "LSMSGFROM":
            msg = "LSMSGFROM Dummy1 T I can only reply with this fixed text message"
        else: 
            msg="Dummy1 recv error" ;  print(msg)  
        return msg
    
    def recv_from_Dummy2():
        if Dummy2_buf[0:7] == "MSGFROM":
            msgs = Dummy2_buf.split(" ",2)
            msg = "MSGFROM Dummy2 "+msgs[2]
        elif Dummy2_buf[0:9] == "BSMSGFROM":
                msgs = Dummy2_buf.split(" ",3)
                msg = "BSMSGFROM Dummy2 "+msgs[2]+" "+msgs[3]           
                # (This is normal string, not bytearray)
        elif Dummy2_buf[0:9] == "LSMSGFROM":
                msgs = Dummy2_buf.split(" ",3)
                msg = "LSMSGFROM Dummy2 "+msgs[2]+" "+msgs[3]           
                # (This is normal string, not bytearray)       
        else:
            msg="Dummy2 recv error" ;  print(msg)
        return msg
    
    def send_to_Dummy1(msg):
        global Dummy1_buf
        Dummy1_buf = msg  # Whole msg including command (Normal str, not bytearray)
        # (Note Clients do not write directly to dummys.  Clients write to Svr & Svr writes to dummys )
        return
    
    def send_to_Dummy2(msg):
        global Dummy2_buf
        Dummy2_buf = msg  # Whole msg incl command.  (Normal str, not a bytearray)   
        # (Note Clients do not write directly to dummys.  Clients write to Svr & Svr writes to dummys )
        return
           
    def from_Print(msg):
        L=len(msg)
        if L >300: 
            msgs = msg[0:300]+"... etc ..."+msg[L-8:L]
        else: msgs = msg
        print("msg received from connection is:",msgs,"(Length=",L,")" )
    
    def to_Print(msg):
        L=len(msg)
        if L>300: 
            msgs = msg[0:300]+"... etc ..."+msg[L-8:L]
        else: msgs = msg
        print("msg sent to connection is:",msgs,"(Length=",L,")" )
                            
    def quit(cuN):
        global whoDB, invitedDB, connectedDB, socketDB
        print("Unregistering (if necessary) & disconnecting ", cuN)
        send_to_client ("You will be unregistered (if necessary) and disconnected from the Svr")
        time.sleep(1.0)  # Give it time (1 s) to send message before socket closes
        if (cuN in whoDB): 
            i = whoDB.index(cuN) 
            del whoDB[i]
            del invitedDB[i]
            del connectedDB[i]
            del socketDB[i]
        clientsocket.close()
        new_cuN = ""
        return new_cuN
    
    def notunderstood(msg):
        if msg.rstrip()!= "" : send_to_client ("Message not understood")
        return
    
    def who(my_cuN):
        if (my_cuN not in whoDB): 
            send_to_client("Your username must be registered")
            return
        msg="Registered:"
        for i in range(0,len(whoDB)): msg += " "+whoDB[i]
        i = whoDB.index(my_cuN)
        msg += " Invited: "+my_cuN+": "+invitedDB[i]
        msg += " Connected: "+my_cuN+": "+connectedDB[i]
        send_to_client(msg)       
        return
    
    def bsparams(my_cuN):
        # BSPARAMS beP plP bfac pakSize
        if (my_cuN not in whoDB): 
            send_to_client("Error: Your username must be registered")
            return
        msgs = msg.split(" ",4)  
        beP = msgs[1] ; framelossP = msgs[2] ; burstFac = msgs[3]; frameSize= msgs[4]
        send_to_client("BSPARAMS: beP="+beP+" framelossP="+framelossP+" burstFac="+burstFac+" frameSize="+frameSize)
        return beP, framelossP, burstFac, frameSize

    def bsinsert_errors(bits_msg, low_beP, high_beP,flossP, burstP):
        # Insert bit errors into bit-string bits_msg for one frame only.  
        # Call with low_beP, high_beP, flossP (frameloss prob) & burstP (burst prob) given as floats.
        L = len(bits_msg)  # Just one frame 
        if np.random.uniform() < flossP:
            new_msg="0" * L     # Fill with "0" characters (lost frame)
            return new_msg
        # Frame is not lost, so we now insert some bit-errors:
        R = np.random.uniform()  # Generate a random number in range 0 to 1 (uniformly distributed)
        if R <= burstP : my_beP = high_beP    # (Make it a burst frame with high biterr_Prob)
        else: my_beP = low_beP                    # (make it a non-burst frame)
        BBA = bytearray(bits_msg, "latin-1")  #  Convert to bitstring to bit-bytearray           
        for i in range(0,L):
            R = np.random.uniform()  # Generate another random number in range 0 to 1
            if R < my_beP :
                if (BBA[i]==48): BBA[i]=49
                elif  (BBA[i]==49): BBA[i]=48
                # Otherwise no effect
        new_msg = BBA.decode("latin-1")
        return new_msg
       
    def msg_method(my_cuN, msg):
        # When Svr receives command MSG from my_cuN, it sends message to 'towhom_cuN': 
        # MSGFROM <my_cuN> msg
        # Waits for reply from towhom.
        # Forwards towhom's reply as return message to my_cuN (or time-out message)
        msgs = msg.split(" ",2)
        towhom = msgs[1]  ;  text = msgs[2].rstrip() 
        print(my_cuN," has sent a message to Svr, intended for ", towhom)
        if (my_cuN not in whoDB): 
            send_to_client(my_cuN+" is not registered - message not sent to Svr")
            return
        # Check if towhom has registered and is connected to clientuserName
        if (towhom not in whoDB):
            send_to_client(towhom+" is not registered - message not sent to Svr")
            return
        i = whoDB.index(my_cuN)  ;  j = whoDB.index(towhom)                   
        if (connectedDB[i] != towhom) and (connectedDB[j] != my_cuN):
            send_to_any_client("Error: "+towhom+" is not connected to "+my_cuN, my_cuN)
            return
        send_to_any_client("MSGFROM "+my_cuN+" "+text, towhom)  # a 'from' message
        print("Msg received by Svr from ",my_cuN," and forwarded to ",towhom,my_cuN)
        print("Waiting for reply from ",towhom," before replying to ",my_cuN)
        if towhom=="Dummy1" or towhom=="Dummy2":
            # Dummys always reply immediately
            ret_msg = receive_from_any_client(towhom)  # Read from Dummy1_buf or Dummy2_buf
            print(towhom, " replied to Svr straightaway. Svr will forward this reply to client")
            send_to_any_client(ret_msg , my_cuN)
            print("Not waiting for a reply from client ",my_cuN)
        return 
    
    def bsmsg(my_cuN, msg):
        # Receives message: BSMSG <towhom_cuN> string
        # Sends message to 'towhom_cuN': BSMSGFROM <my_cuN> bits_string
        # Waits for reply from towhom.
        # Forwards towhom's reply as return message to my_cuN (or time-out message)
        if (my_cuN not in whoDB): 
            send_to_client("Your username must be registered")
            return        
        msgs=msg.split(" ",3)
        # Expects command: BSMSG <towhom> <what> bits_msg    (what = I or S currently)
        towhom = msgs[1]  ;  what = msgs[2];  bits_msg = msgs[3].rstrip()
        # Check if towhom has registered and is connected to clientuserName
        if (towhom not in whoDB):
            send_to_client(towhom+" is not registered")
            return
        i = whoDB.index(my_cuN)  ;  j = whoDB.index(towhom)                   
        if (connectedDB[i] != towhom) and (connectedDB[j] != my_cuN):
            send_to_any_client("Error: "+towhom+" is not connected to "+my_cuN, my_cuN)
            return
        p=0;  pend=0; ps = int(frameSize)  
        Lb = len(bits_msg); new_bits_msg = ""
        # ----- ----- ----- ----- ----- -----
        # Prepare parameters for generating errors:
        # Parameters were entered as strings: beP,  framelossP, burstFac, frameSize
        # Let frames be burst-frames with high_beP or non-burst frames with low_beP
        # When burst-frame probability is zero, low_beP = beP & high_beP doesn't matter
        ratio=100  #  Choose a ratio of high_beP to low_beP for burstiness
        flossP = float(framelossP);  burstP = float(burstFac)
        low_beP = float(beP)/(1.0 + burstP*(ratio-1)) ;  high_beP = ratio*low_beP
        while (pend < Lb) :
            pend=p+ps; 
            if (pend > Lb): pend = Lb
            new_bits_msg += bsinsert_errors(bits_msg[p:pend], low_beP,high_beP,flossP, burstP)
            # NB Works frame by frame, and there's no chance of commands getting corrupted
            p = p + ps  
        # End of while clause
        #----- ----- ----- ----- ----- -----
        send_to_any_client("BSMSGFROM "+my_cuN+" "+what+" "+new_bits_msg, towhom)  # send 'from' msg
        print("Message received from ",my_cuN," by Svr and forwarded to ",towhom,my_cuN)
        print("Svr is waiting for reply from ",towhom," perhaps on other thread to send to ", my_cuN)
        if towhom=="Dummy1" or towhom=="Dummy2":
            # Dummys will reply to Svr straightway
            bits_msg = receive_from_any_client(towhom)  # Received from Dummy1_buf or Dummy2_buf
            from_Print(bits_msg)
            print(towhom, " replied to Svr straightaway. Svr will forward this reply to client")
            send_to_any_client(bits_msg , my_cuN)
        return 
     
    def lsmsg(my_cuN, msg):
        # Receives message: LSMSG <towhom_cuN> <what> normal-string
        # Sends message to 'towhom_cuN': BSMSGFROM <my_cuN> string
        # Waits for reply from towhom.
        # Forwards towhom's reply as return message to my_cuN (or time-out message)
        if (my_cuN not in whoDB): 
            send_to_client("Your username must be registered")
            return        
        msgs=msg.split(" ",3)
        # Expects command: LSMSG <towhom> <what> msg    (what = I or S currently)
        towhom = msgs[1]  ;  what = msgs[2];  my_msg = msgs[3]
        # Check if towhom has registered and is connected to clientuserName
        if (towhom not in whoDB):
            send_to_client(towhom+" is not registered")
            return
        i = whoDB.index(my_cuN)  ;  j = whoDB.index(towhom)                   
        if (connectedDB[i] != towhom) and (connectedDB[j] != my_cuN):
            send_to_any_client("Error: "+towhom+" is not connected to "+my_cuN, my_cuN)
            return 
        send_to_any_client("LSMSGFROM "+my_cuN+" "+what+" "+my_msg, towhom)  # send 'from' msg
        print("Message received from ",my_cuN," by Svr and forwarded to ",towhom,my_cuN)
        print("Svr is waiting for reply from ",towhom," to send to ", my_cuN)
        if towhom=="Dummy1" or towhom=="Dummy2":
            # Dummys will reply to Svr straightway
            ret_msg = receive_from_any_client(towhom)  # Received from Dummy1_buf or Dummy2_buf
            from_Print(ret_msg)
            print(towhom, " replied to Svr straightaway. Svr will forward this reply to client")
            send_to_any_client(ret_msg , my_cuN)
        return  
    
    def send_to_client(msg):
        # msg is a normal Py3 string which is converted to utf-8 (or latin-1) before sending
        # Longer messages will be fragmented automatically by TCP
        if len(msg) > 1024: print("send-message length = ", len(msg))
        clientsocket.sendall(bytes(msg+"\n", "utf-8"))    #  Sending adds newline on end
        
    def send_to_any_client(msg, anyclient):
        # msg is a normal Py3 string which is converted to RTF-8 before sending
        # Longer messages will be fragmented automatically by TCP (hopefully)
        if len(msg) > 1024: print("send-message length = ", len(msg) )
        if anyclient=="Dummy1": send_to_Dummy1(msg )  # Normal str, not bytes
        elif anyclient=="Dummy2": send_to_Dummy2(msg ) # Normal str, not bytes            
        else:
            i = whoDB.index(anyclient)
            socketDB[i].sendall(bytes(msg+"\n","utf-8"))      # Newline added for send
        return
      
    def receive_from_client():
        # Receives utf-8 message & delivers normal Py3 string (msg).  Waits to receive message
        # of any length.  Dont convert to str until end, becos not all utf-8 chars take just 1 byte.
        MTU = 8192;  msg=b""  # Working with bytes, not messages
        while True:
            msg += clientsocket.recv(MTU)
            if msg[-1] == 10 : break  # 10 is code for newline
            # end of while clause
        msg = msg.decode("utf-8")  # Convert bytes to normal string msg
        return msg.rstrip("\n")     # string msg, not bytestring.  Newline on end removed by receiver
        
    def receive_from_any_client(anyclient):
        # Receives utf-8 message & delivers normal Py3 string (msg)
        # Keep receiving until buffer is empty.  Can receive msg of any length.
        if anyclient=="Dummy1": 
            msg = recv_from_Dummy1()   # delivers fixed msg as normal string
            return msg
        if anyclient=="Dummy2": 
            msg = recv_from_Dummy2()  # delivers message from Dummy2_buf
            return msg
        # Wait to receive message from anyclient
        i = whoDB.index(anyclient)
        msg = b""  ;  MTU = 8192   # msg starts off as a bytes object (keep it so till end)
        while True:
            msg += socketDB[i].recv(MTU)  # Receive bytes from real socket
            if msg[-1] ==10 : break    # 10 is code for newline ("\n")
            # end of while clause
        msg = msg.decode("utf-8")  # convert to normal string
        return msg.rstrip("\n")  # (msg is now string) Newline removed by recvr.
    #
    # === === === === === === === === === === === === === === === === ===
    # Start executing the on-new-client method: 
    # === === === === === === === === === === === === === === === === ===
    print("Start executing a version of on-new-client Method on thread :", tid)
    print("socket=",clientsocket)
    print("ip:", addr[0], "port",addr[1]) 
    
    clientuserName = ""  # Will hold the client userName for this thread
    msg = ""
    # Initialise (default) error parameters for BSMSG messages:
    beP = "0";  framelossP = "0" ; burstFac = "0"; frameSize = 2000
    while (msg != "QUIT"):
        msg = receive_from_client()  ;  from_Print(msg)
        # msg = msg.rstrip()
        lock.acquire()  
        if msg[0:8].upper() == "REGISTER" : clientuserName = register(clientuserName, msg)
        elif msg[0:10].upper() == "UNREGISTER": clientuserName=unregister(clientuserName)
        elif msg[0:6].upper() == "INVITE": invite(clientuserName,msg)
        elif msg[0:8].upper() == "UNINVITE": uninvite(clientuserName)
        elif msg[0:11].upper() == "FORCEINVITE" : forceinvite(clientuserName)       
        elif msg[0:6].upper() == "ACCEPT": accept(clientuserName, msg)
        elif msg[0:7].upper() == "DECLINE": decline(clientuserName,msg)
        elif msg[0:3].upper() == "MSG": msg_method(clientuserName,msg)
        elif msg[0:4].upper() == "QUIT": clientuserName = quit(clientuserName)
        elif msg[0:3].upper() == "WHO": who(clientuserName)
    
        #Extra commands for Task 5:
        elif msg[0:8].upper() == "BSPARAMS": beP,framelossP,burstFac,frameSize=bsparams(clientuserName)
        elif msg[0:5].upper() == "BSMSG":  bsmsg(clientuserName,msg)
        elif msg[0:5].upper() == "LSMSG":  lsmsg(clientuserName,msg)
        else: notunderstood(msg)
        #time.sleep(15) #shows how the lock works
        lock.release()
    
    print("End of on_new_client method")        
    return                        
#==== ==== ==== ==== ==== ==== ==== ==== ==== ==== ==== ====                            
# Main program starts here
#==== ==== ==== ==== ==== ==== ==== ==== ==== ==== ==== ====
# DataBase of info that all threads must be able to read & write to without but not at the same time.
# Two dummy clients are always registered.
whoDB  =    ["Dummy1", "Dummy2"] # List of clients that are currently registered
invitedDB = ["none","none"]  # List for storing which (if any) client i is inviting to connect
connectedDB = ["none","none"] # List for storing which client (if any) client i is connected to
socketDB =  ["none","none"] # List of sockets for the clients currently registered in whoDB
Dummy1_buf = ""  ;  Dummy2_buf=""

# We need a lock object for making sure that threads do not try to access the DB lists at the same time.
# All threads will refer to this lock object before they try to read/write the DB lists.
# Whichever thread gets there first will have exclusive access to the DB Lists.  The others will have to wait.

lock = _thread.allocate_lock()  # This is a lock (mutex) object in the main thread.  It is not a thread.

# Now create a socket object (AF_INET= IPv4 ,  SOCK_STREAM = TCP)
S = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
S.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) # Don't have to wait to get port back after a ctrl C
# host = socket.gethostname() # Gets local machine name (but we don not want this right now)
host = 'localhost'  #  Get local host IP address 
port = 9999                # Reserve a port for the service.
print ("Server started and waiting for clients...")
S.bind((host, port))        # Bind to the port
S.listen(5)                 # Alow queue of up to 5 connection requests.
tid = 0   # Thread id
while True:
    # This while loop is in the main thread.  
    # We just go round and round these few statements waiting for a new connection.
    # When we get a connection, we start a new thread which runs an independent version of the method 
    # 'start_new_thread' on that new thread.  
    clientsocket, addr = S.accept()  # Waits here until a new client tries to connect
    tid = tid + 1
    print("Connection from ",addr," has been established.  Starting thread ", id)
    _thread.start_new_thread (on_new_client , (clientsocket, addr, lock, tid))
S.close()


Server started and waiting for clients...
Connection from  ('127.0.0.1', 60203)  has been established.  Starting thread  <built-in function id>
Start executing a version of on-new-client Method on thread : 1
socket= <socket.socket fd=63, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 9999), raddr=('127.0.0.1', 60203)>
ip: 127.0.0.1 port 60203
msg received from connection is: Message to server from MT Client (Length= 32 )
msg received from connection is: REGISTER SP (Length= 11 )
Client username to be registered:  
msg received from connection is: INVITE Dummy2 (Length= 13 )
Dummy2  is automatically ACCEPTING a connection to  SP
msg received from connection is: BSPARAMS 0.000005 0.05 0 3200 (Length= 29 )
msg received from connection is: BSMSG Dummy2 S 000001000000000000000000111111000000010000000100000001000000010011111100111111001111100011111000111110000000010011111100000001001111110000000000111111000000010011111100111111000000000011111100111110